In [3]:
from math import sin, floor

rotl = lambda x, k: ((x << k) | (x >> (32 - k))) & (2**32 - 1)
rotr = lambda x, k: rotl(x, 32 - k)


init = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476]
h = init.copy()
s = (
    [7, 12, 17, 22] * 4
    + [5, 9, 14, 20] * 4
    + [4, 11, 16, 23] * 4
    + [6, 10, 15, 21] * 4
)

f = []
f.append(lambda B, C, D: (B & C) | (~B & D))
f.append(lambda B, C, D: (B & D) | (~D & C))
f.append(lambda B, C, D: B ^ C ^ D)
f.append(lambda B, C, D: C ^ (B | ~D))

w = []
w.append(lambda state, i: state[i])
w.append(lambda state, i: state[(5 * i + 1) % 16])
w.append(lambda state, i: state[(3 * i + 5) % 16])
w.append(lambda state, i: state[7 * i % 16])

t = [floor(2**32 * abs(sin(i))) for i in range(1, 65)]

In [4]:
import struct

def compress(self, block: bytes):
    state = struct.unpack("<16I", block)

    a, b, c, d = self.h

    for i in range(64):
        fi = self.f[i // 16](b, c, d)
        wi = self.w[i // 16](state, i)
        ti = self.t[i]
        si = self.s[i]

        q = b + rotl((a + fi + wi + ti) % 2**32, si)

        a, b, c, d = d, q % 2**32, b, c

    self.h = [(x + y) % 2**32 for x, y in zip([a, b, c, d], self.h)]

In [6]:
import struct
from math import sin, floor

rotl = lambda x, k: ((x << k) | (x >> (32 - k))) & (2**32 - 1)
rotr = lambda x, k: rotl(x, 32 - k)


class MD5:
    def __init__(self):
        self.init = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476]
        self.h = self.init.copy()
        self.s = (
            [7, 12, 17, 22] * 4
            + [5, 9, 14, 20] * 4
            + [4, 11, 16, 23] * 4
            + [6, 10, 15, 21] * 4
        )

        self.f = []
        self.f.append(lambda B, C, D: (B & C) | (~B & D))
        self.f.append(lambda B, C, D: (B & D) | (~D & C))
        self.f.append(lambda B, C, D: B ^ C ^ D)
        self.f.append(lambda B, C, D: C ^ (B | ~D))

        self.w = []
        self.w.append(lambda state, i: state[i])
        self.w.append(lambda state, i: state[(5 * i + 1) % 16])
        self.w.append(lambda state, i: state[(3 * i + 5) % 16])
        self.w.append(lambda state, i: state[7 * i % 16])

        self.t = [floor(2**32 * abs(sin(i))) for i in range(1, 65)]

        self.tail = b""
        self.length = 0

    def update(self, m: bytes):
        curtail = self.tail + m

        while len(curtail) >= 64:
            block = curtail[:64]
            self.length += 512
            self.compress(block)
            curtail = curtail[64:]
        self.tail = curtail

    def compress(self, block: bytes):
        state = struct.unpack("<16I", block)

        a, b, c, d = self.h

        for i in range(64):
            fi = self.f[i // 16](b, c, d)
            wi = self.w[i // 16](state, i)
            ti = self.t[i]
            si = self.s[i]

            q = b + rotl((a + fi + wi + ti) % 2**32, si)

            a, b, c, d = d, q % 2**32, b, c

        self.h = [(x + y) % 2**32 for x, y in zip([a, b, c, d], self.h)]

    def digest(self):
        block = self.tail
        self.length += len(block) * 8
        block += b"\x80"
        while len(block) % 64 != 56:
            block += b"\x00"

        block += struct.pack(b"<Q", self.length)
        self.update(block[len(self.tail):])
        return struct.pack("<IIII", *self.h)

    def hexdigest(self):
        return self.digest().hex()

In [7]:
md5 = MD5()
md5.update(b'aboba')
print(md5.hexdigest())

150f15e73422e0a5ba5b59f997fc2350


In [8]:
from hashlib import md5
md5(b'aboba').hexdigest()

'150f15e73422e0a5ba5b59f997fc2350'

In [9]:
rotl64 = lambda x, n: ((x << n) | x >> (64 - n)) & (2**64 - 1)

def fmix64(k):
    k ^= k >> 33
    k = (k * 0xff51afd7ed558ccd) & (2**64 - 1)
    k ^= k >> 33
    k = (k * 0xc4ceb9fe1a85ec53) & (2**64 - 1)
    k ^= k >> 33
    return k

def MurmurHash3_x64_128 (key, seed=0):
    nblocks = len(key) // 16

    h1 = seed
    h2 = seed

    c1 = 0x87c37b91114253d5;
    c2 = 0x4cf5ad432745937f;


    blocks = [int.from_bytes(key[i: i + 8], 'little') for i in range(0, len(key), 8)]
    for i in range(nblocks):
        k1 = blocks[2 * i]
        k2 = blocks[2 * i + 1]

        k1 = (k1 * c1) & (2**64 - 1)
        k1 = rotl64(k1, 31)
        k1 = (k1 * c2) & (2**64 - 1)
        h1 ^= k1
        h1 = rotl64(h1, 27)
        h1 += h2
        h1 = (h1*5+0x52dce729) & (2**64 - 1)

        k2 = (k2 * c2) & (2**64 - 1)
        k2 = rotl64(k2, 33)
        k2 = (k2 * c1) & (2**64 - 1)
        h2 ^= k2
        h2 = rotl64(h2, 31)
        h2 += h1
        h2 = (h2*5+0x38495ab5) & (2**64 - 1)

    tail = key[nblocks * 16:]

    k1 = 0
    k2 = 0

    if len(tail) > 0:
        k1 = int.from_bytes(tail[:8], 'little')
        k1 = (k1 * c1) & (2**64 - 1)
        k1 = rotl64(k1, 31)
        k1 = (k1 * c2) & (2**64 - 1)
        h1 ^= k1

        k2 = int.from_bytes(tail[8:], 'little')
        k2 = (k2 * c2) & (2**64 - 1)
        k2 = rotl64(k2, 33)
        k2 = (k2 * c1) & (2**64 - 1)
        h2 ^= k2


    h1 ^= len(key)
    h2 ^= len(key)

    h1 += h2
    h2 += h1
    h1 %= 2**64
    h2 %= 2**64


    h1 = fmix64(h1)
    h2 = fmix64(h2)

    h1 += h2
    h2 += h1
    h1 %= 2**64
    h2 %= 2**64

    return hex(h1)[2:].zfill(16) + hex(h2)[2:].zfill(16)